![CH5-ADS.png](.\Media\CH5-ADS.png)

# <span style="color:#cc5500;">DBCC CHECKDB</span>

DBCC CHECKDB checks the logical and physical integrity of all of the objects in a specified database.  [DBCC CHECKDB (Transact-SQL) - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/t-sql/database-console-commands/dbcc-checkdb-transact-sql?view=sql-server-ver15)

- This is the main consistency checking command
- It runs all of the consistency checks against a single database (covered in sections below)
- Very simple to execute:
    - DBCC CHECKDB (N‘database’)
    - Or you can change context to the desired database and just do DBCC CHECKDB
- It is the only way to read all allocated pages in the database
    - It forces all of the page checksums to be checked
- It is possible to change some of the behavior using a variety of options
- It is very resource intensive with CPU, memory, and I/O

It will use multiple threads on the Enterprise Edition of SQL Server

- Parallelism can be disabled using documented trace flag 2528.  This means that CHECKDB will use less resources with parallelism disabled, but the trade-off is that it will take longer go run

## <span style="color:#cc5500;">Other DBCC CHECK Commands</span>

Think of these as sub-commands of DBCC CHECKDB.  They can be run independently of CHECKDB.  The CHECKALLOC, CHECKTABLE, CHECKCATALOG, and CHECKFILEGROUP functionality is covered by DBCC CHECKDB.  It runs this commnands as a part of its process. 

- DBCC CHECKALLOC
    - Checks consistency of allocation structures in a database (GAM pages, SGAM pages, PFS pages, and IM Pages all of with are structures/bitmaps that keep track of which pages and extensts are allocated, and which tables and indexes are allocated as well)
        - After first checking the consistency of critical system tables
    - Examples:
        - Ensure allocation structures in the database are valid
        - Ensure no two tables have the same data file pages allocated
- DBCC CHECKTABLE
    - Checks consistency of a table and its indexes
    - Examples:
        - Ensure each data file page in the table and its indexes can be read and has a valid structure
        - Ensure each nonclustered index row has a matching table row
        - Ensure order of keys in indexes are correct
        - Ensure that any FILESTREAM data is correctly linked to the table
- DBCC CHECKCATALOG
    - Check relationships between system catalogs
        - E.g. column metadata must exist for all columns listed in a table’s metadata, and vice-versa

The following three Check Commands are seperate Check Commands not run by DBCC CHECKDB

- DBCC CHECKFILEGROUP
    - Perform consistency checks for a filegroup
    - Runs allocation checks for the database, then logical consistency checks for table and index partitions in the specified filegroup
    - Simimilar to CHECKDB, but scaled down to just a specified filegroup
- DBCC CHECKIDENT
    - Checks and can reset the identity value for a table
- DBCC CHECKCONSTRAINTS
    - Checks FOREIGN KEY and CHECK constraints for validity
    - Generally, you would want to run this command after a Repair operation.  Covered in a later notebook

## <span style="color:#cc5500;">What exactly does DBCC CHECK do?</span>

- Primitive checks of critical system tables
    - If any problems are found, the only option is to restore from backups or try to extract data manually
- Allocation checks
- Logical checks of critical system tables
- Logical checks of all other tables
- Metadata checks
- Service Broker data validation
- Indexed view, XML index, spatial index checks
    - Only when the WITH EXTENDED\_LOGICAL\_CHECKSoption is used, from SQL Server 2008 onward
- If a repair option is specified, repairs are done at each stage if necessary and possible
- Another way to think of it is that DBCC CHECKDB does:
    - Primitive system table checks, If that passes...then
    - DBCC CHECKALLOC, If that passes...then
    - DBCC CHECKTABLE of system tables, If that passes...then
    - DBCC CHECKTABLE of user tables, If that passes...then
    - DBCC CHECKCATALOG, If that passes...then
    - Service Broker checks and other optional checks
- Because Identity values and Constraint values are relational concepts and not Storage Engine concepts, CHECKDB does not do:
    - DBCC CHECKIDENT
    - DBCC CHECKCONSTRAINTS

## <span style="color:#cc5500;">Demo: Viewing Last Known Good DBCC CHECKDB</span>

This Demo will show you, through the use of an Undocumented DBCC Command called DBCC DBINFO on how to view the Last-Known-Good DBCC CHECKDB

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Demo script for Viewing Last-Known-Good Time demo.

-- Cleanup from the previous notebook demo
USE [master];
GO

IF DATABASEPROPERTYEX (N'Company', N'Version') > 0
BEGIN
	ALTER DATABASE [Company] SET SINGLE_USER
		WITH ROLLBACK IMMEDIATE;
	DROP DATABASE [Company];
END
GO

-- Create a new database
CREATE DATABASE [Company] ON PRIMARY (
    NAME = N'Company',
    FILENAME = N'C:\Program Files\Microsoft SQL Server\MSSQL14.VIPER\MSSQL\DATA\Company.mdf')  --modify path for your environment
LOG ON (
    NAME = N'Company_log',
    FILENAME = N'C:\Program Files\Microsoft SQL Server\MSSQL14.VIPER\MSSQL\DATA\Company_log.ldf'); --modify path for your environment
GO


In [ ]:
-- Now you can Run DBCC CHECKDB on it
DBCC CHECKDB (N'Company') WITH NO_INFOMSGS;
GO


In [ ]:
-- View the last-known-good time.  You turn on Trace 3604 and then it will output the contents of the boot page
--Scroll through the results to see the value dbi_dbccLastKnownGood = <your date>
DBCC TRACEON (3604);
DBCC DBINFO (N'Company');
GO


In [ ]:
-- This is not recommended for Production, but for demonstration purposes only
-- Another option is to clear the error log and the explicitly set your datababase offlline, and then online
EXEC sp_cycle_errorlog;
GO

ALTER DATABASE [Company] SET OFFLINE;
ALTER DATABASE [Company] SET ONLINE;

-- Look in the errorlog and you will see the last-know-good CHECKDB if there was one performed before

## <span style="color:#cc5500;">DBCC CHECKDB Options</span>

- NO\_INFOMSGS
    - Skip printing informational messages
    - Recommended to always use
- ALL\_ERRORMSGS
    - Print all error messages
    - Not required from SQL Server 2008 SP1 onward
- NOINDEX
    - Skip checking nonclustered indexes
    - Not recommended
- DATA\_PURITY
    - Perform column data validation
    - Default for all databases created on SQL Server 2005 or higher
- ESTIMATEONLY
    - Estimate how much tempdb space is required
- TABLOCK
    - Use locks instead of a database snapshot
    - Not permitted on the master database
    - Default for tempdb, where it causes allocation checks to be skipped
- EXTENDED\_LOGICAL\_CHECKS
    - Perform extra validation of some data structures
    - Indexed views, XML indexes, spatial indexes
- PHYSICAL\_ONLY
    - Skips the majority of logical consistency checks
    - Reduces the time necessary for completion
    - Reduces CPU and memory resources necessary

## <span style="color:#cc5500;">Demo the DBCC CHECKDB Options</span>

Prerequisite:  Download the Sample Database named SalesDB from this link: https://www.sqlskills.com/resources/conferences/salesdb2014.zip 

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
-- Setup script for DBCC CHECK Options demo.

-- Download the SalesDB Sample Database from the link above
-- and unzip into one of your drives

-- Restore as follows:
USE [master];
GO

IF DATABASEPROPERTYEX (N'SalesDB', N'Version') > 0
BEGIN
	ALTER DATABASE [SalesDB] SET SINGLE_USER
		WITH ROLLBACK IMMEDIATE;
	DROP DATABASE [SalesDB];
END
GO

RESTORE DATABASE [SalesDB]
FROM DISK = N'C:\Temp\SalesDB2014.bak'  --modify path for your environment
WITH
    MOVE N'SalesDBData' TO N'C:\Program Files\Microsoft SQL Server\MSSQL14.VIPER\MSSQL\DATA\SalesDB.mdf',  --modify path for your environment
	MOVE N'SalesDBLog' TO N'C:\Program Files\Microsoft SQL Server\MSSQL14.VIPER\MSSQL\DATA\SalesDB_log.ldf';  --modify path for your environment
GO


In [ ]:
-- Cleanup from previous notebook
USE [master];
GO

IF DATABASEPROPERTYEX (N'Company', N'Version') > 0
BEGIN
	ALTER DATABASE [Company] SET SINGLE_USER
		WITH ROLLBACK IMMEDIATE;
	DROP DATABASE [Company];
END
GO

-- Create a new database
CREATE DATABASE [Company] ON PRIMARY (
    NAME = N'Company',
    FILENAME = N'C:\Program Files\Microsoft SQL Server\MSSQL14.VIPER\MSSQL\DATA\Company.mdf')  --modify path for your environment
LOG ON (
    NAME = N'Company_log',
    FILENAME = N'C:\Program Files\Microsoft SQL Server\MSSQL14.VIPER\MSSQL\DATA\Company_log.ldf'); --modify path for your environment
GO

In [ ]:
-- Run DBCC CHECKDB on it with no options
-- More will be covered on this in a more advanced notebook, but notice all of the output
DBCC CHECKDB (N'Company');
GO

In [ ]:
-- Remove all the informational output and just let me know if there are any corruptions
DBCC CHECKDB (N'Company') WITH NO_INFOMSGS;
GO

In [ ]:
-- Estimate of tempdb space that will be used by CHECKDB
DBCC CHECKDB (N'Company') WITH ESTIMATEONLY;
GO

In [ ]:
-- Now, let's run this against the SaleDB

-- WARNING!  It is not recommended that you run DBCC DROPCLEANBUFFERS on a production system.  
-- This DBCC Drop clean buffer operation removes all the buffers from the buffer cache which contain the data already moved to the disk. In other words, 
-- this operation flushes out all the clean pages (which were dirty before CHECKPOINT executed) out of the memory.
-- Make note of the time and resources used

DBCC DROPCLEANBUFFERS;
GO
SET STATISTICS TIME ON;
GO
DBCC CHECKDB (N'SalesDB') WITH NO_INFOMSGS;
GO
SET STATISTICS TIME OFF;
GO

In [ ]:
-- Do Physical Only
-- Note that it takes less time and fewer resources for this limited check
DBCC DROPCLEANBUFFERS;
GO
SET STATISTICS TIME ON;
GO
DBCC CHECKDB (N'SalesDB') WITH NO_INFOMSGS, PHYSICAL_ONLY;
GO
SET STATISTICS TIME OFF;
GO

## <span style="color:#cc5500;">How long will it take to run DBCC CHECKDB?</span>

The answer to that depends on many factors:

- Size of the database.  The more allocated pages there are, the longer it takes
- Concurrent I/O load on the server
- Concurrent CPU activity on the server
- Concurrent update activity on the database
    - As mentioned above, DBCC CHECKDB makes use of snapshots.  If there are a lot of database updates occuring, the snapshot has to keep up with those updates.
- Throughput capabilities of the I/O subsystem
- Number of CPUs on the server
- Speed of the disks where tempdb is placed
    - tempdb is used to store intermediate data that CHECKDB uses
- Complexity of the database schema
- Which options were specified when you ran CHECKDB
- Number and type of corruptions that exist

## <span style="color:#cc5500;">Breaking up Consistency Checks over Time</span>

I you have a large database and DBCC CHECKDB simply take too long to run, you can break your consistency checks into chunks that will take less time.

- Several options for doing the equivalent of DBCC CHECKDB if it’s not possible to restore to another server and run DBCC CHECKDB there
    - E.g. no other system to use or the database is too large
- Option 1:
    - Use DBCC CHECKFILEGROUP to perform per-filegroup checks of all filegroups over a number of days, plus DBCC CHECKCATALOG
    - This is another benefit of using multiple filegroups when your database size becomes very large (e.g. more than 100 GB)
- Option 2:
    - Use DBCC CHECKTABLE to perform consistency checks of all tables over a number of days, plus DBCC CHECKALLOCand DBCC CHECKCATALOG
    - Can be based on time, table size, or some other factor